In [140]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [141]:
# restore all of our data structures
import pickle
tf.reset_default_graph()
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [142]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [143]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w.lower() == s.lower(): 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [144]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'age_neg', u'age_pos', u'facebook', u'goodbye', u'grade', u'home', u'location', u'name', u'pets', u'subscribe', u'youtube']


In [145]:
# load our saved model

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /home/salts/cybersecurity-exhibition/chatbot/model.tflearn


In [151]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.05
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    print(results)
    # if we have a classification then find the matching intent tag
    
    
    
    if (userID in context and context[userID] is not None):
        for j in results:
            for i in intents["intents"]:
                if i['tag'] == j[0] and 'context_filter' in i and i["context_filter"] == context[userID]:
                    return random.choice(i["responses"])
                    
        
        for i in intents['intents']:
            if ('context_filter' in i and i["context_filter"] == context[userID]):
                if ('context_set' in i):
                    context[userID] = i['context_set']
                else:
                    context[userID] = None
                return random.choice(i["responses"])
    
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    contextValue = None
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        contextValue = i['context_set']
                        

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        if (contextValue is not None):
                            context[userID] = i['context_set']
                        return random.choice(i['responses'])
                    
                    if (contextValue is not None):
                        context[userID] = i['context_set']

            results.pop(0)

In [152]:
print(response('hei'))

[(u'name', 0.89588857)]
Tere, mis su nimi on? 😀😀😀


In [155]:
context = {'123': None}

print(response("hei"))
print(context)
print(response("Ei"))

[(u'name', 0.89588857)]
Tere, mis su nimi on? 😀😀😀
{'123': u'name'}
[(u'pets', 0.16835), (u'age_neg', 0.1664725), (u'home', 0.1638602), (u'subscribe', 0.13951814), (u'youtube', 0.12742971), (u'facebook', 0.09275214), (u'grade', 0.056854986), (u'location', 0.05627906)]
match
Okei, Ma olen Kadri, kohe saan 12. Kui vana sa praegu oled?


In [81]:

context

{'123': u'name'}

In [82]:
print(response('Peeter'))

[]
Ma olen Mari ja olen 14-aastane. Kui vana sa oled?


In [83]:
context

{'123': u'age'}

In [84]:
print(response('15'))

[]
Lahe! Mul on kodus kass ja ma elan ühes korteris Mustamäel. Kus sa elad?


In [85]:
print(response("Tallinnas"))

[(u'pets', 0.89581215)]
Okei! Kus su kodu muidu asub?


In [86]:
print(response("Pelgulinnas"))

[]
Okei :) Kas sul ka lemmikloomi on?


In [87]:
print(response("jah, koer"))

[]
Tead, mul on üks hästi lahe YouTube’i konto, ma panen sinna iga nädal ühe video üles. Kas sul ka seal konto on? Anna oma kasutajanimi ja ma Subscriben!


In [88]:
print(response("nimi123"))

[]
Okei! Kuule, mis su Facebook on? Ma tahaks sind sõbraks lisada!


In [89]:
print(response("nimi123"))

[]
Ok :) Kuule, mis klassis sa käisidki?


In [90]:
print(response("3"))

[]
Ok :) ! Kuule, ma pean praegu minema, räägime teine kord uuesti! Tsau!


In [31]:
classify('today')

[]

In [32]:
# clear context
response("Hi there!", show_details=True)

('tag:', u'goodbye')
Nägemist
('context:', u'home')
('context:', u'age')


In [33]:
response('today')
classify('today')

[]

In [34]:
response("thanks, your great")